# Step 1: fine-tune LLM using top result from (fixed) ranker

In [2]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    os.environ['BATCH_SIZE'] = '1'
    os.environ['r'] = os.environ.get('r', '10')
    os.environ['k'] = os.environ.get('k', '2')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 1 max_iteration = 5 model_type = xxl *********
n                 iter (since)         2 loss (since)       2 rouge1 (since)   2 rouge1 ema (since) 2 rouge1 (dev) (since)      dt
1                0.000 (0.000)          0.260 (0.260)          0.359 (0.359)          0.359 (0.359)          0.000 (0.000)  17.1 s
2                0.000 (0.000)          0.160 (0.060)          0.367 (0.375)          0.367 (0.375)          0.000 (0.000)    37 s
4                0.000 (0.000)          0.085 (0.010)          0.663 (0.958)          0.653 (0.940)          0.000 (0.000)   1.2 m
8                0.000 (0.000)          0.751 (1.417)          0.428 (0.194)          0.477 (0.300)          0.000 (0.000)  2.28 m
16               0.000 (0.000)          0.864 (0.977)          0.532 (0.637)          0.539 (0.602)          0.000 (0.000)  4.36 m
32               0.000 (0.000)          0.999 (1.135)          0.512 (0.491)          0.569 (0.599)          0.000 (0.000)  8.54 m
64               

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    os.environ['BATCH_SIZE'] = '2'
    os.environ['LEARN_BATCH_SIZE'] = '2'
    os.environ['GRAD_FREE_BATCH_SIZE'] = '16'
    os.environ['r'] = os.environ.get('r', '1')
    os.environ['k'] = os.environ.get('k', '2')
    os.environ['STEP1_ITER'] = os.environ.get('STEP1_ITER', '3_augment1')
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission(*, step2_iter, step1_iter, k):
    import evaluate
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNews import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xxl', load_in_8bit=True)
    taskllm_model_id = f'User_keq{k}_t5xxl_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'raw_taskllm')
    t5.load_adapter(taskllm_model_id, 'ema_taskllm')
    rhat_model_id = f'User_keq{k}_t5xxl_step2_iter{step2_iter}'
    t5.load_adapter(rhat_model_id, 'raw_rhat')
    t5.load_adapter(rhat_model_id, 'ema_rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_suffix="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_suffix="rhat", model_id=rhat_model_id)
    rouge_metric = evaluate.load('rouge')
    gradfree_batch_size = 128
    n_randos = 128

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0]-1, 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        safek = min(k, scores.shape[0])
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=safek).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]
    
    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} ***')

    devgolds, testgolds = [], []
    with ProgressPrinter(f'{k} rouge (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['article'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = gradfree_batch_size,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], n_randos) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = gradfree_batch_size,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                greedyaction = [ rhats[a:b].argmax().item() for a, b in zip(splits, splits[1:]) ]
                greedyprompts = [ prompt[a] for prompt, a in zip(prompts, greedyaction) ]
                guesses = sum(inner_batch(func = taskllm.generate,
                                          inner_batch_size = gradfree_batch_size,
                                          inputs = (greedyprompts,)
                                         ),
                              [])
                if isdev:
                    rewards = sum( ( rouge_metric.compute(predictions=[guess],
                                                          references=[label],
                                                          use_aggregator=False)['rouge1']
                                     for guess, label in zip(guesses, labels)
                                  ),
                                  [])
                    rewards = torch.Tensor(rewards, device='cpu').mean().item()
                else:
                    rewards = None
                
                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': guess })

            printer.addobs(rewards)

    for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
        with open(f'lamp4u_{wut}golds_t5xxl_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}.json', 'w') as jsonfile:
            json.dump({ 'task': 'LaMP_4', 'golds': golds }, jsonfile)
            
prepare_submission(k=4, step1_iter='0_augment2', step2_iter='0_augment1')